# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [105]:
import $ivy.`org.scalaj::scalaj-http:2.4.2`

import $ivy.$                              

In [106]:
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [107]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

In [108]:
import $ivy.`org.json4s::json4s-jackson:3.4.0`
import org.json4s.JsonDSL._
import org.json4s._
import org.json4s.jackson.JsonMethods._
implicit val formats = DefaultFormats

import $ivy.$                                 

import org.json4s.JsonDSL._

import org.json4s._

import org.json4s.jackson.JsonMethods._

formats: DefaultFormats.type = org.json4s.DefaultFormats$@1065e3fa

In [109]:
import scala.collection.mutable.ListBuffer

import scala.collection.mutable.ListBuffer

### 1.1 Obtener el texto del RSS Feed y REDDIT Feed

Creamos una clase la cual toma un argumento que definira como se usara.
Consta de un metodo queryURL(url: String) ---> Seq[String] el cual recibe una url y devuelve una lista de texto, la cual tiene los datos de dicha url.

Para RSS:

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

Para REDDIT:

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse.Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML.Luego, se parsea con JSON para extraer los campos `title` y `selftext`.

In [110]:
val url1 = "https://www.reddit.com/r/Android/hot/.json?count=10"
val url2 = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

url1: String = "https://www.reddit.com/r/Android/hot/.json?count=10"
url2: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

In [111]:
class GetTextURL(urlType: String){
    // Obtener texto desde una url
    def queryURL(url: String): Seq[String] = {
        try{
            val response: HttpResponse[String] = Http(url)
              .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
              .asString
            val stringBody = response.body
            urlType match {
                case "rss" => {
                    val xml = XML.loadString(stringBody)
                    // Extract text from title and description
                    (xml \\ "item").map { item => ((item \ "title").text + " " + (item \ "description").text) }
                }
                case "reddit" => {
                    // parse Reddit feed in JSON
                    val result = (parse(stringBody) \ "data" \ "children" \ "data")
                         .extract[List[Map[String, Any]]]
                    // Parsear JSON
                    val filterContent = result.flatten.filter{case (v , _) => v == "title" || v == "selftext" }.map(x => x._2.toString)
                    val pattern = "(https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]".r
                    filterContent.map(x => pattern.replaceAllIn(x,"")).toSeq
                }
            }
        }catch{
            case e: Exception => List()
        }    
    }
}

defined class GetTextURL

In [114]:
val rss = new GetTextURL("rss")
val rssT = rss.queryURL(url2)

rss: GetTextURL = ammonite.$sess.cmd110$Helper$GetTextURL@5c4f583b
rssT: Seq[String] = List(
  "Andrew Vaughn\u2019s 1st major-league home run helps power the Chicago White Sox to their 5th straight win: \u2018Pretty special moment. It came at a good time, too.\u2019 Andrew Vaughn hit the first home run of his major-league career in the Chicago White Sox's 13-8 victory against the Minnesota Twins at Guaranteed Rate Field. The Sox won their fifth straight, and it's their fourth consecutive game scoring at least nine runs.",
  "The highs and lows of the 2020-21 Chicago Blackhawks: From Jonathan Toews\u2019 season-long absence to Patrick Kane\u2019s milestone and Kirby Dach\u2019s injury and return The Chicago Blackhawks' 2020-21 season started with offseason news about Kirby Dach but held a few other surprises. Here's a timeline of their year.",
  "The Chicago Bears\u2019 2021 schedule is out. Here are our game-by-game predictions. The Chicago Bears have the third-toughest schedule in th

In [115]:
val reddit = new GetTextURL("reddit")
val redditT = reddit.queryURL(url1)

reddit: GetTextURL = ammonite.$sess.cmd110$Helper$GetTextURL@38676b12
redditT: Seq[String] = List(
  """**Credits to the team at /r/PickAnAndroidForMe for compiling this information:**

* Home - 

* Smartphones 101 - 

* Top Phones - 


***        
Note 1. Join us at /r/MoronicMondayAndroid, a sub serving as a repository for our retired weekly threads. Just pick any thread and Ctrl-F your way to wisdom! 

Note 2. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]()""",
  "What should I buy Thursday (May 13 2021) - Your weekly device inquiry thread!",
  """Hey /r/android, we had conducted a [feedback poll]() last year. We have decided to make a few revisions (mainly a preferential voting system) since some of the results were rather open-ended and did not present a clear solution. Participation on the last poll was also lower than anticipated since we only got about [930 responses]().

NOTES

* The poll was created via Google Forms and requires sign-in to pre

## 2. Detectar las entidades nombradas

### 2.1 Crear la clase encargada del modelo,contar,y ordenar Entidades

Los metodos de esta clase son :

getNEs(textList: Seq[String]) ---> Seq[Seq[String]] el cual que recibe una lista de textos y aplica el modelo, para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

count(textL : Seq[Seq[String]]) ---> Map[String,Int] el cual recibe el resultado de aplicar el modelo, devuelve la entidad junto las cantidad de veces que aparece nombrada

sort(counts: Map[String,Int]) ---> List[(String, Int)] el cual ordena el resultado de contar las entidades

In [116]:
class NERModel() {
    // Variables Necesarias para crear modelo
    private val STOPWORDS = Seq (
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
        "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
        "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
        "their", "theirs", "themselves", "what", "which", "who", "whom",
        "this", "that", "these", "those", "am", "is", "are", "was", "were",
        "be", "been", "being", "have", "has", "had", "having", "do", "does",
        "did", "doing", "a", "an", "the", "and", "but", "if", "or",
        "because", "as", "until", "while", "of", "at", "by", "for", "with",
        "about", "against", "between", "into", "through", "during", "before",
        "after", "above", "below", "to", "from", "up", "down", "in", "out",
        "off", "over", "under", "again", "further", "then", "once", "here",
        "there", "when", "where", "why", "how", "all", "any", "both", "each",
        "few", "more", "most", "other", "some", "such", "no", "nor", "not",
        "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
        "will", "just", "don", "should", "now", "on",
        // Contractions without '
        "im", "ive", "id", "Youre", "youd", "youve",
        "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
        "theyre", "theyd", "theyve",
        "shouldnt", "couldnt", "musnt", "cant", "wont",
        // Common uppercase words
        "hi", "hello"
    )
    private val punctuationSymbols = ".,()!?;:'`´\n"
    private val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Aplicar el Modelo a los datos (simplemente es aplicar la funcion a la lista de textos)
    private def getNEsSingle(text: String): Seq[String] =
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq

    def getNEs(textList: Seq[String]): Seq[Seq[String]] = textList.map(getNEsSingle)
    
    // Contar las entidades
    def count(textL : Seq[Seq[String]]): Map[String,Int] = {
            textL.flatten.foldLeft(Map.empty[String, Int]) {
             (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    //Ordenar Entidades
    def sort(counts: Map[String,Int]): List[(String, Int)] = {
        counts.toList
          .sortBy(_._2)(Ordering[Int].reverse)
    }
}
       

defined class NERModel

In [117]:
val model = new NERModel

model: NERModel = ammonite.$sess.cmd115$Helper$NERModel@2ad8afa0

### 2.2 Aplicar el "Modelo" a los datos

In [118]:
val result_rss = model.getNEs(rssT)
val result_reddit = model.getNEs(redditT)

result_rss: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Andrew",
    "Vaughn\u2019s",
    "Chicago",
    "White",
    "Sox",
    "Andrew",
    "Vaughn",
    "Chicago",
    "White",
    "Soxs",
    "Minnesota",
    "Twins",
    "Guaranteed",
    "Rate",
    "Field",
    "Sox"
  ),
  ArrayBuffer(
    "Chicago",
    "Blackhawks",
    "Jonathan",
    "Toews\u2019",
    "Patrick",
    "Kane\u2019s",
    "Kirby",
    "Dach\u2019s",
    "Chicago",
    "Blackhawks",
    "Kirby",
    "Dach",
    "Heres"
  ),
  ArrayBuffer(
    "Chicago",
    "Bears\u2019",
    "Chicago",
    "Bears",
...
result_reddit: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Home",
    "Smartphones",
    "Top",
    "Phones",
    "Note",
    "Join",
    "Ctrl-F",
    "Note",
    "Join",
    "IRC",
    "Telegram"
  ),
  ArrayBuffer("Thursday", "May"),
  ArrayBuffer(
    "Hey",
    "Participation",
    "Google",
    "Forms",
    "Email",
    "Responses",
    "Well",
    "POLL]Edit"
  ),
  ArrayBuffer("Community", "Feedback",

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [119]:
val count_rss = model.count(result_rss)
val count_reddit = model.count(result_reddit)

count_rss: Map[String, Int] = Map(
  "Rate" -> 4,
  "University" -> 1,
  "Nets" -> 3,
  "Gleyber" -> 1,
  "Spirit\u2019s" -> 1,
  "Corey" -> 2,
  "Marisnick" -> 1,
  "Polisky" -> 4,
  "Operations" -> 1,
  "Cub" -> 1,
  "Parker" -> 4,
  "Crawford" -> 2,
  "Bieber" -> 2,
  "Sky" -> 3,
  "Hawaii" -> 2,
  "Washington" -> 1,
  "IL" -> 1,
  "Robert" -> 1,
  "Dach\u2019s" -> 1,
  "Hockey" -> 1,
  "DeBrincat\u2019s" -> 1,
  "Marisnick\u2019s" -> 2,
  "President" -> 1,
  "Alex" -> 1,
  "Pat" -> 2,
  "Abreu\u2019s" -> 1,
  "Colt" -> 2,
  "Shane" -> 2,
  "Finnish" -> 1,
  "COVID" -> 2,
  "Henrik" -> 2,
  "Toews\u2019" -> 2,
  "ERA" -> 1,
  "Bortles" -> 1,
  "Baldwin" -> 2,
  "Andrew" -> 2,
  "MVP" -> 1,
  "Phase" -> 1,
...
count_reddit: Map[String, Int] = Map(
  "Much" -> 1,
  "Participation" -> 1,
  "Well" -> 1,
  "Feedback" -> 1,
  "February" -> 1,
  "Central-" -> 1,
  "Zenfone" -> 8,
  "Goggles" -> 1,
  "Xperia" -> 1,
  "IP68" -> 1,
  "PowerShell" -> 1,
  "Look" -> 1,
  "Comparison" -> 2,
  "P

In [120]:
val sort_rss = model.sort(count_rss)
val sort_reddit = model.sort(count_reddit)

sort_rss: List[(String, Int)] = List(
  ("Chicago", 49),
  ("White", 12),
  ("Sox", 12),
  ("Blackhawks", 11),
  ("Minnesota", 8),
  ("Twins", 8),
  ("NFL", 8),
  ("Cubs", 7),
  ("Bears", 7),
  ("Tuesday", 6),
  ("Bulls", 6),
  ("Photos", 6),
  ("Wednesday", 5),
  ("Patrick", 5),
  ("Rate", 4),
  ("Polisky", 4),
  ("Parker", 4),
  ("Mike", 4),
  ("Guaranteed", 4),
  ("Candace", 4),
  ("Field", 4),
  ("Cleveland", 4),
  ("Indians", 4),
  ("Nets", 3),
  ("Sky", 3),
  ("Brooklyn", 3),
  ("Alzolay", 3),
  ("Milwaukee", 3),
  ("May", 3),
  ("Stan", 3),
  ("Former", 3),
  ("COVID-19", 3),
  ("Here\u2019s", 3),
  ("Zach", 3),
  ("Northwestern", 3),
  ("Soxs", 3),
  ("Bowman", 3),
  ("Jonathan", 3),
...
sort_reddit: List[(String, Int)] = List(
  ("Flip", 10),
  ("Google", 9),
  ("Zenfone", 8),
  ("ZenFone", 6),
  ("Asus", 4),
  ("Android", 4),
  ("Note", 3),
  ("ASUS", 3),
  ("Camera", 3),
  ("Comparison", 2),
  ("US", 2),
  ("Hey", 2),
  ("Review", 2),
  ("Realme", 2),
  ("Phone", 2),
  ("Sam

## 4. FeedService


In [121]:
class FeedService(){
    //Variable donde almacenar las suscripciones
    private val buffer = new ListBuffer[(String,GetTextURL)]()
    
    //Guardar un registro de las URL, y opcionalmente sus parámetros, suscriptas.
    def suscribe(url: String, t: GetTextURL) : Unit= {
        buffer += ((url,t))
    }
    
    //Obtener los feeds
    def get_feed(): Seq[Seq[String]] = {
        val model = new NERModel
        val toSeq = buffer.toSeq
        toSeq.map{ x => x._2.queryURL(x._1)}
    }
    
    //Compilar el resultado de cada una en una única lista
    def get_result(feed: Seq[Seq[String]]): Seq[String] = {
        feed.flatten
    }
    
}

defined class FeedService

In [122]:
val feed_service = new FeedService()

feed_service: FeedService = ammonite.$sess.cmd120$Helper$FeedService@1fdc8c5e

In [123]:
feed_service.suscribe(url1, new GetTextURL("reddit"))
feed_service.suscribe(url1, new GetTextURL("reddit"))
feed_service.suscribe(url2, new GetTextURL("rss"))
feed_service.suscribe(url1, new GetTextURL("reddit"))

In [124]:
val get_feed = feed.get_feed

get_feed: Seq[Seq[String]] = List(
  List(
    """**Credits to the team at /r/PickAnAndroidForMe for compiling this information:**

* Home - 

* Smartphones 101 - 

* Top Phones - 


***        
Note 1. Join us at /r/MoronicMondayAndroid, a sub serving as a repository for our retired weekly threads. Just pick any thread and Ctrl-F your way to wisdom! 

Note 2. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]()""",
    "What should I buy Thursday (May 13 2021) - Your weekly device inquiry thread!",
    """Hey /r/android, we had conducted a [feedback poll]() last year. We have decided to make a few revisions (mainly a preferential voting system) since some of the results were rather open-ended and did not present a clear solution. Participation on the last poll was also lower than anticipated since we only got about [930 responses]().

NOTES

* The poll was created via Google Forms and requires sign-in to prevent duplication of votes.

* Most questions requi

In [125]:
val get_result = feed.get_result(get_feed)

get_result: Seq[String] = List(
  """**Credits to the team at /r/PickAnAndroidForMe for compiling this information:**

* Home - 

* Smartphones 101 - 

* Top Phones - 


***        
Note 1. Join us at /r/MoronicMondayAndroid, a sub serving as a repository for our retired weekly threads. Just pick any thread and Ctrl-F your way to wisdom! 

Note 2. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]()""",
  "What should I buy Thursday (May 13 2021) - Your weekly device inquiry thread!",
  """Hey /r/android, we had conducted a [feedback poll]() last year. We have decided to make a few revisions (mainly a preferential voting system) since some of the results were rather open-ended and did not present a clear solution. Participation on the last poll was also lower than anticipated since we only got about [930 responses]().

NOTES

* The poll was created via Google Forms and requires sign-in to prevent duplication of votes.

* Most questions require a response alt

In [126]:
val model_N = new NERModel

model_N: NERModel = ammonite.$sess.cmd115$Helper$NERModel@f270a92

In [127]:
val do_model = model_N.getNEs(get_result)
val do_count = model_N.count(do_model)
val do_sort = model_N.sort(do_count)

do_model: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Home",
    "Smartphones",
    "Top",
    "Phones",
    "Note",
    "Join",
    "Ctrl-F",
    "Note",
    "Join",
    "IRC",
    "Telegram"
  ),
  ArrayBuffer("Thursday", "May"),
  ArrayBuffer(
    "Hey",
    "Participation",
    "Google",
    "Forms",
    "Email",
    "Responses",
    "Well",
    "POLL]Edit"
  ),
  ArrayBuffer("Community", "Feedback", "Poll", "February"),
  ArrayBuffer(),
  ArrayBuffer("Google", "Messages"),
  ArrayBuffer(),
  ArrayBuffer(),
  ArrayBuffer(),
  ArrayBuffer("US", "Agrees", "Remove", "Xiaomi", "Blacklist", "Lawsuit"),
  ArrayBuffer(),
  ArrayBuffer("Hey", "Google"),
  ArrayBuffer(),
  ArrayBuffer("Samsung", "Galaxy", "Watch", "WearOS"),
  ArrayBuffer(
    "RedditASUS",
    "Europe",
...
do_count: Map[String, Int] = Map(
  "Much" -> 3,
  "Rate" -> 4,
  "University" -> 1,
  "Participation" -> 3,
  "Nets" -> 3,
  "Gleyber" -> 1,
  "Spirit\u2019s" -> 1,
  "Well" -> 3,
  "Corey" -> 2,
  "Marisnick" -> 1,
 